# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1] #This accesses the second table (since Python indexing starts at 0) from the list of DataFrames returned by pd.read_html().
cities=cities.iloc[:-1,[0,3,5,6,7,8]] #The -1 index means "exclude the last row", the latter means select columns at index positions 0, 3, 5, 6, 7, and 8.

def nhl_correlation(): 
    # YOUR CODE HERE
    # Step 1: Filter the data just to get 2018 data
    nhl_dfcopy = nhl_df.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    nhl_dfcopy = nhl_dfcopy[nhl_dfcopy.year == 2018]

    nhlcitiescopy = cities.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    
    # Step 2: Discard unncessary columns
    nhl_dfcopy = nhl_dfcopy[['team', 'W', 'L']]
    
    # Step 3: Clean the team names, some team names having "*" at the end
    def clean_star(team):
        if "*" in team:
            return re.findall("([\w\s]*)\*", team)[0]
        else:
            return team
    nhl_dfcopy['team'] = nhl_dfcopy.team.apply(clean_star)
    #nhl_dfcopy.to_excel('NHL.xlsx') #saved at the /assignments/course1_assignment4/ folder
    
    
    # Step 4: Clean the team names in the cities df, some teams having '[]' in them
    def clean_bracket(team):
        if "[" in team:
            return re.findall("([\w\s]*)\[", team)[0]
        else:
            return team
    nhlcitiescopy['NHL'] = nhlcitiescopy['NHL'].apply(clean_bracket)
    #nhlcitiescopy.to_excel('NHL_CITIES.xlsx') #saved at the /assignments/course1_assignment4/ folder
    
    # Step 4.1: Discard the unnecessary columns to remove NFL, MLB, NBA. Only Keep NHL & Let's rename cities column in more readable format
    nhlcitiescopy = nhlcitiescopy[['Metropolitan area', 'Population (2016 est.)[8]','NHL']]
    nhlcitiescopy.rename(columns = {'Metropolitan area': 'City', 'Population (2016 est.)[8]': 'Population','NHL': 'team_only'}, inplace = True)
    
    # Step 5: in the nhl_df, the team name columns contain city names also, need to seperate them out. 
    nhl_dfcopy['team_only'] = nhl_dfcopy['team'].apply(lambda x: x.rsplit(None, 1)[-1]) #This lambda function will not work for those some teams e.g.: Toronto Maple Leafs -> Maple Leafs
    
    nhl_dfcopy.loc[3, 'team_only'] = 'Maple Leafs' #so have to do manually
    nhl_dfcopy.loc[5, 'team_only'] = 'Red Wings'
    nhl_dfcopy.loc[13, 'team_only'] = 'Blue Jackets'
    nhl_dfcopy.loc[27, 'team_only'] = 'Golden Knights'
    
    nhl_dfcopy = nhl_dfcopy[~nhl_dfcopy.W.str.contains('Division')] #Remove unncessary rows with 'Division' word in it in either column WIN OR LOSE.
    
    # Step 5.1: replace the names of teams of the same city e.g. new york city/new jersey have 3 teams: Rangers, Islanders, Devils
    nhl_dfcopy.loc[14, 'team_only'] = 'Rangers Islanders Devils' #so have to do manually
    nhl_dfcopy.loc[16, 'team_only'] = 'Rangers Islanders Devils'
    nhl_dfcopy.loc[17, 'team_only'] = 'Rangers Islanders Devils'
    
    nhl_dfcopy.loc[28, 'team_only'] = 'Kings Ducks' #so have to do manually
    nhl_dfcopy.loc[30, 'team_only'] = 'Kings Ducks'
    
    #Step 6: Change the datatypes of win and lose to float
    nhl_dfcopy.W = nhl_dfcopy.W.astype(float)
    nhl_dfcopy.L = nhl_dfcopy.L.astype(float)
    
    # Step 7: Since there are 4 teams with the same name, lets group them and get their average win, loss values
    nhl_dfcopy = nhl_dfcopy.groupby(by = 'team_only')['W', 'L'].mean()
    
    # Step 8: Reset the index because after grouping by 'team_only', the index become 'team_only', we want to keep our original default index
    nhl_dfcopy.reset_index(inplace = True)
    
    # Step 9: Merge the 2 dataframes 
    merged_df = pd.merge(nhl_dfcopy, nhlcitiescopy, how = 'inner', on = 'team_only')
    # merged_df.dtypes to check the datatypes is correct
    
    # Step 10: Create the column win_loss_ratio = win/ win+lose
    merged_df['win_lose_ratio'] = merged_df.W / (merged_df.W + merged_df.L)
    
    # Step 11: Finally pass the numberic values of population_by_region and the win_loss_by_region
    population_by_region = pd.to_numeric(merged_df['Population'], errors = 'coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df['win_lose_ratio'], errors = 'coerce') # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    #'coerce': If any value can't be converted to a number, it will be replaced with NaN (Not a Number).
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0] #It returns 2 values-tuple, but we only want the first value.
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    raise NotImplementedError()

In [2]:
nhl_correlation()

0.01230899645574427

In [40]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1] #This accesses the second table (since Python indexing starts at 0) from the list of DataFrames returned by pd.read_html().
cities=cities.iloc[:-1,[0,3,5,6,7,8]] #The -1 index means "exclude the last row", the latter means select columns at index positions 0, 3, 5, 6, 7, and 8.

def nhl_output(): 
    # YOUR CODE HERE
    # Step 1: Filter the data just to get 2018 data
    nhl_dfcopy = nhl_df.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    nhl_dfcopy = nhl_dfcopy[nhl_dfcopy.year == 2018]

    nhlcitiescopy = cities.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    
    # Step 2: Discard unncessary columns
    nhl_dfcopy = nhl_dfcopy[['team', 'W', 'L']]
    
    # Step 3: Clean the team names, some team names having "*" at the end
    def clean_star(team):
        if "*" in team:
            return re.findall("([\w\s]*)\*", team)[0]
        else:
            return team
    nhl_dfcopy['team'] = nhl_dfcopy.team.apply(clean_star)
    #nhl_dfcopy.to_excel('NHL.xlsx') #saved at the /assignments/course1_assignment4/ folder
    
    
    # Step 4: Clean the team names in the cities df, some teams having '[]' in them
    def clean_bracket(team):
        if "[" in team:
            return re.findall("([\w\s]*)\[", team)[0]
        else:
            return team
    nhlcitiescopy['NHL'] = nhlcitiescopy['NHL'].apply(clean_bracket)
    #nhlcitiescopy.to_excel('NHL_CITIES.xlsx') #saved at the /assignments/course1_assignment4/ folder
    
    # Step 4.1: Discard the unnecessary columns to remove NFL, MLB, NBA. Only Keep NHL & Let's rename cities column in more readable format
    nhlcitiescopy = nhlcitiescopy[['Metropolitan area', 'Population (2016 est.)[8]','NHL']]
    nhlcitiescopy.rename(columns = {'Metropolitan area': 'City', 'Population (2016 est.)[8]': 'Population','NHL': 'team_only'}, inplace = True)
    
    # Step 5: in the nhl_df, the team name columns contain city names also, need to seperate them out. 
    nhl_dfcopy['team_only'] = nhl_dfcopy['team'].apply(lambda x: x.rsplit(None, 1)[-1]) #This lambda function will not work for those some teams e.g.: Toronto Maple Leafs -> Maple Leafs
    
    nhl_dfcopy.loc[3, 'team_only'] = 'Maple Leafs' #so have to do manually
    nhl_dfcopy.loc[5, 'team_only'] = 'Red Wings'
    nhl_dfcopy.loc[13, 'team_only'] = 'Blue Jackets'
    nhl_dfcopy.loc[27, 'team_only'] = 'Golden Knights'
    
    nhl_dfcopy = nhl_dfcopy[~nhl_dfcopy.W.str.contains('Division')] #Remove unncessary rows with 'Division' word in it in either column WIN OR LOSE.
    
    # Step 5.1: replace the names of teams of the same city e.g. new york city/new jersey have 3 teams: Rangers, Islanders, Devils
    nhl_dfcopy.loc[14, 'team_only'] = 'Rangers Islanders Devils' #so have to do manually
    nhl_dfcopy.loc[16, 'team_only'] = 'Rangers Islanders Devils'
    nhl_dfcopy.loc[17, 'team_only'] = 'Rangers Islanders Devils'
    
    nhl_dfcopy.loc[28, 'team_only'] = 'Kings Ducks' #so have to do manually
    nhl_dfcopy.loc[30, 'team_only'] = 'Kings Ducks'
    
    #Step 6: Change the datatypes of win and lose to float
    nhl_dfcopy.W = nhl_dfcopy.W.astype(float)
    nhl_dfcopy.L = nhl_dfcopy.L.astype(float)
    
    # Step 7: Since there are 4 teams with the same name, lets group them and get their average win, loss values
    nhl_dfcopy = nhl_dfcopy.groupby(by = 'team_only')['W', 'L'].mean()
    
    # Step 8: Reset the index because after grouping by 'team_only', the index become 'team_only', we want to keep our original default index
    nhl_dfcopy.reset_index(inplace = True)
    
    # Step 9: Merge the 2 dataframes 
    merged_df = pd.merge(nhl_dfcopy, nhlcitiescopy, how = 'inner', on = 'team_only')
    # merged_df.dtypes to check the datatypes is correct
    
    # Step 10: Create the column win_loss_ratio = win/ win+lose
    merged_df['win_lose_ratio'] = merged_df.W / (merged_df.W + merged_df.L)
    
    return merged_df[['City', 'win_lose_ratio']]

In [41]:
nhl_output()

,City,win_lose_ratio
0,Denver,0.589041
1,Chicago,0.458333
2,Columbus,0.600000
3,St. Louis,0.578947
4,Boston,0.714286
5,Montreal,0.420290
6,Vancouver,0.436620
7,"Washington, D.C.",0.653333
8,Phoenix,0.414286
9,Calgary,0.513889


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    # Step 1: Filter the data just to get 2018 data
    nba_dfcopy = nba_df.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    nba_dfcopy = nba_dfcopy[nba_dfcopy.year == 2018]

    nbacitiescopy = cities.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    

    # Step 2: Discard unncessary columns
    nba_dfcopy = nba_dfcopy[['team', 'W', 'L']]
    
    # Step 3: Clean the team names, some team names having "*" and "()" at the end
    nba_dfcopy['team'] = nba_dfcopy.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if "*" in team else team) #this format using lambda more readable
    nba_dfcopy['team'] = nba_dfcopy.team.apply(lambda team: re.findall("([\w\s]*)\(", team)[0] if "(" in team else team)
    
    # Step 4: Clean the team names in the cities df, some teams having '[]' in them
    nbacitiescopy['NBA'] = nbacitiescopy['NBA'].apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if "[" in team else team)
    
    # Step 5: Discard the unnecessary columns to remove NFL, MLB, NHL. Only Keep NBA & Let's rename cities column in more readable format
    nbacitiescopy = nbacitiescopy[['Metropolitan area', 'Population (2016 est.)[8]','NBA']]
    nbacitiescopy.rename(columns = {'Metropolitan area': 'City', 'Population (2016 est.)[8]': 'Population','NBA': 'team_only'}, inplace = True)
    
    # Step 5: in the nba_df, the team name columns contain city names also, need to seperate them out. 
    nba_dfcopy['team_only'] = nba_dfcopy['team'].apply(lambda x: x.rsplit(None, 1)[-1]) #This lambda function will not work for those some teams e.g.: 
    #17 --> Portland Trail Blazers
    
    nba_dfcopy.loc[17, 'team_only'] = 'Trail Blazers' #so have to do manually
    
    # Step 5.1: replace the names of teams of the same city e.g. LA Lakers, LA Clippers, New York Knicks, Brooklyn Nets
    nba_dfcopy.loc[10, 'team_only'] = 'Knicks Nets' #so have to do manually
    nba_dfcopy.loc[11, 'team_only'] = 'Knicks Nets'
    nba_dfcopy.loc[24, 'team_only'] = 'Lakers Clippers'
    nba_dfcopy.loc[25, 'team_only'] = 'Lakers Clippers'
    
    #nba_dfcopy.to_excel('NBA.xlsx') #saved at the /assignments/course1_assignment4/ folder
    #nbacitiescopy.to_excel('NBA_City.xlsx')  #saved at the /assignments/course1_assignment4/ folder
    
    #Step 6: Change the datatypes of win and lose to float
    nba_dfcopy.W = nba_dfcopy.W.astype(float)
    nba_dfcopy.L = nba_dfcopy.L.astype(float)
    
    # Step 7: Since there are 4 teams with the same name, lets group them and get their average win, loss values
    nba_dfcopy = nba_dfcopy.groupby(by = 'team_only')['W', 'L'].mean()
    
    # Step 8: Reset the index because after grouping by 'team_only', the index become 'team_only', we want to keep our original default index
    nba_dfcopy.reset_index(inplace = True)
    
    # Step 9: Merge the 2 dataframes 
    merged_df = pd.merge(nba_dfcopy, nbacitiescopy, how = 'inner', on = 'team_only')
    # merged_df.dtypes to check the datatypes is correct
    
    # Step 10: Create the column win_loss_ratio = win/ win+lose
    merged_df['win_lose_ratio'] = merged_df.W / (merged_df.W + merged_df.L)
    
    # Step 11: Finally pass the numberic values of population_by_region and the win_loss_by_region
    population_by_region = pd.to_numeric(merged_df['Population'], errors = 'coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df['win_lose_ratio'], errors = 'coerce') # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
    #merged_df.to_excel('Merged_NBA.xlsx') #saved at the /assignments/course1_assignment4/ folder
   
    return stats.pearsonr(population_by_region, win_loss_by_region)[0] #It returns 2 values-tuple, but we only want the first value.
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    raise NotImplementedError()
    

In [4]:
nba_correlation()

-0.17657160252844614

In [42]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_output():
    # YOUR CODE HERE
    # Step 1: Filter the data just to get 2018 data
    nba_dfcopy = nba_df.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    nba_dfcopy = nba_dfcopy[nba_dfcopy.year == 2018]

    nbacitiescopy = cities.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    

    # Step 2: Discard unncessary columns
    nba_dfcopy = nba_dfcopy[['team', 'W', 'L']]
    
    # Step 3: Clean the team names, some team names having "*" and "()" at the end
    nba_dfcopy['team'] = nba_dfcopy.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if "*" in team else team) #this format using lambda more readable
    nba_dfcopy['team'] = nba_dfcopy.team.apply(lambda team: re.findall("([\w\s]*)\(", team)[0] if "(" in team else team)
    
    # Step 4: Clean the team names in the cities df, some teams having '[]' in them
    nbacitiescopy['NBA'] = nbacitiescopy['NBA'].apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if "[" in team else team)
    
    # Step 5: Discard the unnecessary columns to remove NFL, MLB, NHL. Only Keep NBA & Let's rename cities column in more readable format
    nbacitiescopy = nbacitiescopy[['Metropolitan area', 'Population (2016 est.)[8]','NBA']]
    nbacitiescopy.rename(columns = {'Metropolitan area': 'City', 'Population (2016 est.)[8]': 'Population','NBA': 'team_only'}, inplace = True)
    
    # Step 5: in the nba_df, the team name columns contain city names also, need to seperate them out. 
    nba_dfcopy['team_only'] = nba_dfcopy['team'].apply(lambda x: x.rsplit(None, 1)[-1]) #This lambda function will not work for those some teams e.g.: 
    #17 --> Portland Trail Blazers
    
    nba_dfcopy.loc[17, 'team_only'] = 'Trail Blazers' #so have to do manually
    
    # Step 5.1: replace the names of teams of the same city e.g. LA Lakers, LA Clippers, New York Knicks, Brooklyn Nets
    nba_dfcopy.loc[10, 'team_only'] = 'Knicks Nets' #so have to do manually
    nba_dfcopy.loc[11, 'team_only'] = 'Knicks Nets'
    nba_dfcopy.loc[24, 'team_only'] = 'Lakers Clippers'
    nba_dfcopy.loc[25, 'team_only'] = 'Lakers Clippers'
    
    #nba_dfcopy.to_excel('NBA.xlsx') #saved at the /assignments/course1_assignment4/ folder
    #nbacitiescopy.to_excel('NBA_City.xlsx')  #saved at the /assignments/course1_assignment4/ folder
    
    #Step 6: Change the datatypes of win and lose to float
    nba_dfcopy.W = nba_dfcopy.W.astype(float)
    nba_dfcopy.L = nba_dfcopy.L.astype(float)
    
    # Step 7: Since there are 4 teams with the same name, lets group them and get their average win, loss values
    nba_dfcopy = nba_dfcopy.groupby(by = 'team_only')['W', 'L'].mean()
    
    # Step 8: Reset the index because after grouping by 'team_only', the index become 'team_only', we want to keep our original default index
    nba_dfcopy.reset_index(inplace = True)
    
    # Step 9: Merge the 2 dataframes 
    merged_df = pd.merge(nba_dfcopy, nbacitiescopy, how = 'inner', on = 'team_only')
    # merged_df.dtypes to check the datatypes is correct
    
    # Step 10: Create the column win_loss_ratio = win/ win+lose
    merged_df['win_lose_ratio'] = merged_df.W / (merged_df.W + merged_df.L)
    
    return merged_df[['City', 'win_lose_ratio']]
    

In [43]:
nba_output()

,City,win_lose_ratio
0,Philadelphia,0.634146
1,Milwaukee,0.536585
2,Chicago,0.329268
3,Cleveland,0.609756
4,Boston,0.670732
5,Memphis,0.268293
6,Atlanta,0.292683
7,Miami–Fort Lauderdale,0.536585
8,Charlotte,0.439024
9,Salt Lake City,0.585366


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    # Step 1: Filter the data just to get 2018 data
    mlb_dfcopy = mlb_df.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    mlb_dfcopy = mlb_dfcopy[mlb_dfcopy.year == 2018]

    mlbcitiescopy = cities.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    
    # Step 2: Discard unncessary columns
    mlb_dfcopy = mlb_dfcopy[['team', 'W', 'L']]
    
    # Step 4: Clean the team names in the cities df, some teams having '[]' in them
    mlbcitiescopy['MLB'] = mlbcitiescopy['MLB'].apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if "[" in team else team)
    
    # Step 5: Discard the unnecessary columns to remove NFL, NBA, NHL. Only Keep MLB & Let's rename cities column in more readable format
    mlbcitiescopy = mlbcitiescopy[['Metropolitan area', 'Population (2016 est.)[8]','MLB']]
    mlbcitiescopy.rename(columns = {'Metropolitan area': 'City', 'Population (2016 est.)[8]': 'Population','MLB': 'team_only'}, inplace = True)
    
    # Step 5: in the nba_df, the team name columns contain city names also, need to seperate them out. 
    mlb_dfcopy['team_only'] = mlb_dfcopy['team'].apply(lambda x: x.rsplit(None, 1)[-1]) #This lambda function will not work for those some teams e.g.: 
    
    # Step 5.1: replace the names of teams of the same city e.g.
    mlb_dfcopy.loc[13, 'team_only'] = 'Dodgers Angels' #so have to do manually
    mlb_dfcopy.loc[25, 'team_only'] = 'Dodgers Angels'
    
    mlb_dfcopy.loc[18, 'team_only'] = 'Yankees Mets'
    mlb_dfcopy.loc[1, 'team_only'] = 'Yankees Mets'
    
    mlb_dfcopy.loc[11, 'team_only'] = 'Giants Athletics'
    mlb_dfcopy.loc[28, 'team_only'] = 'Giants Athletics'
    
    mlb_dfcopy.loc[21, 'team_only'] = 'Cubs White Sox'
    mlb_dfcopy.loc[8, 'team_only'] = 'Cubs White Sox'
    
    mlb_dfcopy.loc[0, 'team_only'] = 'Red Sox'
    mlb_dfcopy.loc[3, 'team_only'] = 'Blue Jays'
    
    #mlb_dfcopy.to_excel('MLB.xlsx') #saved at the /assignments/course1_assignment4/ folder
    #mlbcitiescopy.to_excel('MLB_City.xlsx')  #saved at the /assignments/course1_assignment4/ folder
    
    #Step 6: Change the datatypes of win and lose to float
    mlb_dfcopy.W = mlb_dfcopy.W.astype(float)
    mlb_dfcopy.L = mlb_dfcopy.L.astype(float)
    
    # Step 7: Since there are 4 teams with the same name, lets group them and get their average win, loss values
    mlb_dfcopy = mlb_dfcopy.groupby(by = 'team_only')['W', 'L'].mean()
    
    # Step 8: Reset the index because after grouping by 'team_only', the index become 'team_only', we want to keep our original default index
    mlb_dfcopy.reset_index(inplace = True)
    
    # Step 9: Merge the 2 dataframes 
    merged_df = pd.merge(mlb_dfcopy, mlbcitiescopy, how = 'inner', on = 'team_only')
    # merged_df.dtypes to check the datatypes is correct
    
    # Step 10: Create the column win_loss_ratio = win/ win+lose
    merged_df['win_lose_ratio'] = merged_df.W / (merged_df.W + merged_df.L)
    
    # Step 11: Finally pass the numberic values of population_by_region and the win_loss_by_region
    population_by_region = pd.to_numeric(merged_df['Population'], errors = 'coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df['win_lose_ratio'], errors = 'coerce') # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    
    #merged_df.to_excel('Merged_MLB.xlsx') #saved at the /assignments/course1_assignment4/ folder
   
    return stats.pearsonr(population_by_region, win_loss_by_region)[0] #It returns 2 values-tuple, but we only want the first value.
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    raise NotImplementedError()

    

In [6]:
mlb_correlation()

0.15052304487104848

In [44]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_output(): 
    # YOUR CODE HERE
    # Step 1: Filter the data just to get 2018 data
    mlb_dfcopy = mlb_df.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    mlb_dfcopy = mlb_dfcopy[mlb_dfcopy.year == 2018]

    mlbcitiescopy = cities.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    
    # Step 2: Discard unncessary columns
    mlb_dfcopy = mlb_dfcopy[['team', 'W', 'L']]
    
    # Step 4: Clean the team names in the cities df, some teams having '[]' in them
    mlbcitiescopy['MLB'] = mlbcitiescopy['MLB'].apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if "[" in team else team)
    
    # Step 5: Discard the unnecessary columns to remove NFL, NBA, NHL. Only Keep MLB & Let's rename cities column in more readable format
    mlbcitiescopy = mlbcitiescopy[['Metropolitan area', 'Population (2016 est.)[8]','MLB']]
    mlbcitiescopy.rename(columns = {'Metropolitan area': 'City', 'Population (2016 est.)[8]': 'Population','MLB': 'team_only'}, inplace = True)
    
    # Step 5: in the nba_df, the team name columns contain city names also, need to seperate them out. 
    mlb_dfcopy['team_only'] = mlb_dfcopy['team'].apply(lambda x: x.rsplit(None, 1)[-1]) #This lambda function will not work for those some teams e.g.: 
    
    # Step 5.1: replace the names of teams of the same city e.g.
    mlb_dfcopy.loc[13, 'team_only'] = 'Dodgers Angels' #so have to do manually
    mlb_dfcopy.loc[25, 'team_only'] = 'Dodgers Angels'
    
    mlb_dfcopy.loc[18, 'team_only'] = 'Yankees Mets'
    mlb_dfcopy.loc[1, 'team_only'] = 'Yankees Mets'
    
    mlb_dfcopy.loc[11, 'team_only'] = 'Giants Athletics'
    mlb_dfcopy.loc[28, 'team_only'] = 'Giants Athletics'
    
    mlb_dfcopy.loc[21, 'team_only'] = 'Cubs White Sox'
    mlb_dfcopy.loc[8, 'team_only'] = 'Cubs White Sox'
    
    mlb_dfcopy.loc[0, 'team_only'] = 'Red Sox'
    mlb_dfcopy.loc[3, 'team_only'] = 'Blue Jays'
    
    #mlb_dfcopy.to_excel('MLB.xlsx') #saved at the /assignments/course1_assignment4/ folder
    #mlbcitiescopy.to_excel('MLB_City.xlsx')  #saved at the /assignments/course1_assignment4/ folder
    
    #Step 6: Change the datatypes of win and lose to float
    mlb_dfcopy.W = mlb_dfcopy.W.astype(float)
    mlb_dfcopy.L = mlb_dfcopy.L.astype(float)
    
    # Step 7: Since there are 4 teams with the same name, lets group them and get their average win, loss values
    mlb_dfcopy = mlb_dfcopy.groupby(by = 'team_only')['W', 'L'].mean()
    
    # Step 8: Reset the index because after grouping by 'team_only', the index become 'team_only', we want to keep our original default index
    mlb_dfcopy.reset_index(inplace = True)
    
    # Step 9: Merge the 2 dataframes 
    merged_df = pd.merge(mlb_dfcopy, mlbcitiescopy, how = 'inner', on = 'team_only')
    # merged_df.dtypes to check the datatypes is correct
    
    # Step 10: Create the column win_loss_ratio = win/ win+lose
    merged_df['win_lose_ratio'] = merged_df.W / (merged_df.W + merged_df.L)
    
    return merged_df[['City', 'win_lose_ratio']]

In [45]:
mlb_output()

,City,win_lose_ratio
0,Houston,0.635802
1,Toronto,0.450617
2,Atlanta,0.555556
3,Milwaukee,0.588957
4,St. Louis,0.543210
5,Chicago,0.483077
6,Phoenix,0.506173
7,Los Angeles,0.529231
8,San Francisco Bay Area,0.524691
9,Cleveland,0.561728


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    # Step 1: Filter the data just to get 2018 data
    nfl_dfcopy = nfl_df.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    nfl_dfcopy = nfl_dfcopy[nfl_dfcopy.year == 2018]

    nflcitiescopy = cities.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    
    # Step 2: Discard unnecessary columns
    nfl_dfcopy = nfl_dfcopy[['team', 'W', 'L']]
    
    # Step 2.1: Discard unnecessary rows with 'AFC' or 'NFC' word in it
    nfl_dfcopy = nfl_dfcopy[~nfl_dfcopy.W.str.contains('|'.join(['AFC', 'NFC']))]
    
    # Step 3: Clean the team names, some team names having "*" and "+" at the end
    nfl_dfcopy['team'] = nfl_dfcopy.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if "*" in team else team) #this format using lambda more readable
    nfl_dfcopy['team'] = nfl_dfcopy.team.apply(lambda team: re.findall("([\w\s]*)\+", team)[0] if "+" in team else team)
    
    # Step 4: Clean the team names in the cities df, some teams having '[]' in them
    nflcitiescopy['NFL'] = nflcitiescopy['NFL'].apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if "[" in team else team)
    
    # Step 5: Discard the unnecessary columns to remove NFL, NBA, NHL. Only Keep MLB & Let's rename cities column in more readable format
    nflcitiescopy = nflcitiescopy[['Metropolitan area', 'Population (2016 est.)[8]','NFL']]
    nflcitiescopy.rename(columns = {'Metropolitan area': 'City', 'Population (2016 est.)[8]': 'Population','NFL': 'team_only'}, inplace = True)
    
    # Step 5: in the nba_df, the team name columns contain city names also, need to seperate them out. 
    nfl_dfcopy['team_only'] = nfl_dfcopy['team'].apply(lambda x: x.rsplit(None, 1)[-1]) #This lambda function will not work for those some teams e.g.: 
    
    # Step 5.1: replace the names of teams of the same city e.g.
    nfl_dfcopy.loc[17, 'team_only'] = 'Rams Chargers' #so have to do manually
    nfl_dfcopy.loc[36, 'team_only'] = 'Rams Chargers'
    
    nfl_dfcopy.loc[24, 'team_only'] = 'Giants Jets'
    nfl_dfcopy.loc[4, 'team_only'] = 'Giants Jets'
    
    nfl_dfcopy.loc[19, 'team_only'] = '49ers Raiders'
    nfl_dfcopy.loc[38, 'team_only'] = '49ers Raiders'
    
    #nfl_dfcopy.to_excel('NFL.xlsx') #saved at the /assignments/course1_assignment4/ folder
    #nflcitiescopy.to_excel('NFL_City.xlsx')  #saved at the /assignments/course1_assignment4/ folder
    
    #Step 6: Change the datatypes of win and lose to float
    nfl_dfcopy.W = nfl_dfcopy.W.astype(float)
    nfl_dfcopy.L = nfl_dfcopy.L.astype(float)
    
    # Step 7: Since there are 4 teams with the same name, lets group them and get their average win, loss values
    nfl_dfcopy = nfl_dfcopy.groupby(by = 'team_only')['W', 'L'].mean()
    
    # Step 8: Reset the index because after grouping by 'team_only', the index become 'team_only', we want to keep our original default index
    nfl_dfcopy.reset_index(inplace = True)
    
    # Step 9: Merge the 2 dataframes 
    merged_df = pd.merge(nfl_dfcopy, nflcitiescopy, how = 'inner', on = 'team_only')
    # merged_df.dtypes to check the datatypes is correct
    
    # Step 10: Create the column win_loss_ratio = win/ win+lose
    merged_df['win_lose_ratio'] = merged_df.W / (merged_df.W + merged_df.L)
    
    # Step 11: Finally pass the numberic values of population_by_region and the win_loss_by_region
    population_by_region = pd.to_numeric(merged_df['Population'], errors = 'coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df['win_lose_ratio'], errors = 'coerce') # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    #merged_df.to_excel('Merged_NFL.xlsx') #saved at the /assignments/course1_assignment4/ folder
    
    # Step 11: Finally pass the numberic values of population_by_region and the win_loss_by_region
    population_by_region = pd.to_numeric(merged_df['Population'], errors = 'coerce') # pass in metropolitan area population from cities
    win_loss_by_region = pd.to_numeric(merged_df['win_lose_ratio'], errors = 'coerce') # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0] #It returns 2 values-tuple, but we only want the first value.
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    raise NotImplementedError()
   


In [8]:
nfl_correlation()

0.00492211214934943

In [46]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_output(): 
    # YOUR CODE HERE
    # Step 1: Filter the data just to get 2018 data
    nfl_dfcopy = nfl_df.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    nfl_dfcopy = nfl_dfcopy[nfl_dfcopy.year == 2018]

    nflcitiescopy = cities.copy() #You can create copies of all the global variables with the copy() method and proceed as usual.
    
    # Step 2: Discard unnecessary columns
    nfl_dfcopy = nfl_dfcopy[['team', 'W', 'L']]
    
    # Step 2.1: Discard unnecessary rows with 'AFC' or 'NFC' word in it
    nfl_dfcopy = nfl_dfcopy[~nfl_dfcopy.W.str.contains('|'.join(['AFC', 'NFC']))]
    
    # Step 3: Clean the team names, some team names having "*" and "+" at the end
    nfl_dfcopy['team'] = nfl_dfcopy.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if "*" in team else team) #this format using lambda more readable
    nfl_dfcopy['team'] = nfl_dfcopy.team.apply(lambda team: re.findall("([\w\s]*)\+", team)[0] if "+" in team else team)
    
    # Step 4: Clean the team names in the cities df, some teams having '[]' in them
    nflcitiescopy['NFL'] = nflcitiescopy['NFL'].apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if "[" in team else team)
    
    # Step 5: Discard the unnecessary columns to remove NFL, NBA, NHL. Only Keep MLB & Let's rename cities column in more readable format
    nflcitiescopy = nflcitiescopy[['Metropolitan area', 'Population (2016 est.)[8]','NFL']]
    nflcitiescopy.rename(columns = {'Metropolitan area': 'City', 'Population (2016 est.)[8]': 'Population','NFL': 'team_only'}, inplace = True)
    
    # Step 5: in the nba_df, the team name columns contain city names also, need to seperate them out. 
    nfl_dfcopy['team_only'] = nfl_dfcopy['team'].apply(lambda x: x.rsplit(None, 1)[-1]) #This lambda function will not work for those some teams e.g.: 
    
    # Step 5.1: replace the names of teams of the same city e.g.
    nfl_dfcopy.loc[17, 'team_only'] = 'Rams Chargers' #so have to do manually
    nfl_dfcopy.loc[36, 'team_only'] = 'Rams Chargers'
    
    nfl_dfcopy.loc[24, 'team_only'] = 'Giants Jets'
    nfl_dfcopy.loc[4, 'team_only'] = 'Giants Jets'
    
    nfl_dfcopy.loc[19, 'team_only'] = '49ers Raiders'
    nfl_dfcopy.loc[38, 'team_only'] = '49ers Raiders'
    
    #nfl_dfcopy.to_excel('NFL.xlsx') #saved at the /assignments/course1_assignment4/ folder
    #nflcitiescopy.to_excel('NFL_City.xlsx')  #saved at the /assignments/course1_assignment4/ folder
    
    #Step 6: Change the datatypes of win and lose to float
    nfl_dfcopy.W = nfl_dfcopy.W.astype(float)
    nfl_dfcopy.L = nfl_dfcopy.L.astype(float)
    
    # Step 7: Since there are 4 teams with the same name, lets group them and get their average win, loss values
    nfl_dfcopy = nfl_dfcopy.groupby(by = 'team_only')['W', 'L'].mean()
    
    # Step 8: Reset the index because after grouping by 'team_only', the index become 'team_only', we want to keep our original default index
    nfl_dfcopy.reset_index(inplace = True)
    
    # Step 9: Merge the 2 dataframes 
    merged_df = pd.merge(nfl_dfcopy, nflcitiescopy, how = 'inner', on = 'team_only')
    # merged_df.dtypes to check the datatypes is correct
    
    # Step 10: Create the column win_loss_ratio = win/ win+lose
    merged_df['win_lose_ratio'] = merged_df.W / (merged_df.W + merged_df.L)
    
    return merged_df[['City', 'win_lose_ratio']]


In [47]:
nfl_output()

,City,win_lose_ratio
0,San Francisco Bay Area,0.250000
1,Chicago,0.750000
2,Cincinnati,0.375000
3,Buffalo,0.375000
4,Denver,0.375000
5,Cleveland,0.466667
6,Tampa Bay Area,0.312500
7,Phoenix,0.187500
8,Kansas City,0.750000
9,Indianapolis,0.625000


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [67]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


# YOUR CODE HERE
nhl_table = nhl_output()
nba_table = nba_output()
mlb_table = mlb_output()
nfl_table = nfl_output()
     
def choose_sport(sport):
    if sport == 'NHL':
        return nhl_table
    elif sport == 'NBA':
        return nba_table
    elif sport == 'MLB':
        return mlb_table
    elif sport == 'NFL':
        return nfl_table
    else:
        print("Error with input!")

def sports_team_performance():
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in sports:
        for j in sports:
            if i != j:
                # Merge the data based on 'City' for each sport pair
                sport_i_data = choose_sport(i)
                sport_j_data = choose_sport(j)
                # Merge the two DataFrames on 'City' column
                merged_df = pd.merge(sport_i_data, sport_j_data, how = 'inner', on ='City', suffixes = (f'_{i}', f'_{j}'))
                
                # Perform a paired t-test on the win-lose ratios (assuming the column is 'win_lose_ratio')
                t_stat, p_stat = stats.ttest_rel(merged_df[f'win_lose_ratio_{i}'], merged_df[f'win_lose_ratio_{j}'])
                
                # Store the p-value in the DataFrame
                p_values.loc[i, j] = p_stat
    
    return p_values
                                 
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    
    raise NotImplementedError()

In [68]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
